In [26]:
import webbrowser
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random 

In [27]:
model = load_model('model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('texts.pkl', 'rb'))
classes = pickle.load(open('labels.pkl', 'rb'))  

In [28]:
lemmatizer = WordNetLemmatizer()


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)
        

In [29]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list        

In [30]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result  

    
    return "Sorry, I'm not sure I understand. Can you try asking in a different way?" 


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res  




In [31]:
from flask import Flask, render_template, request


app = Flask(__name__)
app.static_folder = 'static'


@app.route("/")
def home():
    return render_template("index.html")


@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)


if __name__ == "__main__":
    app.run()  

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/May/2024 17:43:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2024 17:43:15] "GET /static/styles/style.css HTTP/1.1" 404 -
127.0.0.1 - - [02/May/2024 17:43:16] "GET /static/images/4.png HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2024 17:43:18] "GET /static/images/3.png HTTP/1.1" 304 -


1/1 [==============================] - 0s 64ms/step


127.0.0.1 - - [02/May/2024 17:43:19] "GET /get?msg=hihih HTTP/1.1" 200 -


1/1 [==============================] - 0s 47ms/step


...

127.0.0.1 - - [02/May/2024 17:43:51] "GET /get?msg=h%20r%20y HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


127.0.0.1 - - [02/May/2024 17:44:03] "GET /get?msg=jeevan HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2024 17:44:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2024 17:44:07] "GET /static/styles/style.css HTTP/1.1" 404 -
127.0.0.1 - - [02/May/2024 17:44:07] "GET /static/images/4.png HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2024 17:44:11] "GET /static/images/3.png HTTP/1.1" 304 -


1/1 [==============================] - 0s 32ms/step


127.0.0.1 - - [02/May/2024 17:44:11] "GET /get?msg=hii HTTP/1.1" 200 -
